<a id = "pymc3"></a>

### Extension: MCMC Techniques with `pymc3`

`PyMC` is a useful module to know for reasons beyond Bayesian Linear Regression, given its implementation of MCMC.   


Instead of solving our Bayesian Regression problem directly, `pymc3` instead uses Markov Chain Monte Carlo techniques to estimate regression parameters.  

You may notice different results between this technique and the solutions found above; much of those differences come from the definition of priors.  

#### `pymc3`
Two examples are given below.

First, a synthetic data-set is created, where a linear relationship is explicitly defined, and noise is added. This should give a good idea of what the parameter estimation in `pymc3` looks like.   

Second, parameters are estimated for the "housing" dataset.  

#### Reading in Housing Data:

In [ ]:
### This cell imports the necessary modules and sets a few plotting parameters for display

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)  

In [ ]:
### Read in the data
tr_path = '../resource/asnlib/publicdata/train.csv'
data = pd.read_csv(tr_path)  

### The .head() function shows the first few lines of data for perspecitve
data.head()

In [ ]:
data.plot('GrLivArea', 'SalePrice', kind = 'scatter', marker = 'x');

In [ ]:
data.plot('YearBuilt', 'SalePrice', kind = 'scatter', marker = 'x');

#### Creating Data

In [ ]:
### Synthetic data

np.random.seed(1738)

# Create True parameters:
intercept, sigma, betas = 20, .5, np.array([0.2, .4])

# set number of observations
obs = 100

predictors = np.random.randn(obs, 2)

# Defining our explicit linear relationship. Notice how the noise is added with `sigma`
Y = intercept + np.matmul(predictors, betas) + np.random.randn(obs)*sigma


# visualize
# A view of our syntetic data, plotting both predictors against the actual values
plt.figure(figsize = (10,4))
plt.subplot(1,2,1)
plt.scatter(predictors[:,0], Y)
plt.title("X1 Predictor")
plt.ylabel("Y")
plt.subplot(1,2,2)
plt.scatter(predictors[:,1],Y)
plt.title("X2 Predictor")

#### Using Housing Data

In [ ]:
import pymc3 as pm

# instantiate basic model
basic_model = pm.Model()

# Define two numpy arrays of predictors
X1 = predictors[:,0]
X2 = predictors[:,1]

# Tell pymc3 that everything is happening within that model
with basic_model:

    # Priors for unknown model parameters
    # Define prior for intercept as observed mean
    intercept = pm.Normal('intercept', mu=np.mean(Y), sd=10)
    
    # note shape = 2 parameter
    # prior for predictors is "0"
    beta = pm.Normal('beta', mu=0, sd=1, shape=2)
    
    # Explicitly define prior for "noise" term
    sigma = pm.Normal('sigma',mu = 1, sd=1)
    
    
    # Expected relationship between variables
    mu = intercept + beta[0]*X1 + beta[1]*X2

    # Likelihood of observations
    # (sampling distribution)
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=Y)
    
    # instantiate sampler
    # `NUTS` is "no u-turn sampler", the most advanced `pymc3` offers
    step = pm.NUTS()

    # draw 500, the 5000 posterior samples
    trace_500_ex = pm.sample(500, step=step)
    trace_5k_ex = pm.sample(5000, step = step)

#### Plotting the distributions of the posteriors, with both 500 and 5,000 draws.

Notice how close in both the 500 draw and 5,000 draw versions the mean "peak" of the parameters is to the actual parameters.

In [ ]:
pm.traceplot(trace_500_ex);

In [ ]:
pm.traceplot(trace_5k_ex);

#### Finding the MAP solution

Once again, close to actual parameters

In [ ]:
pm.find_MAP(model = basic_model)

#### Fitting model to housing data

In [ ]:
house_model = pm.Model()
y = data.SalePrice.values
X = data[['GrLivArea','YearBuilt']].values
X1 = X[:,0]
X2 = X[:,1]

with house_model:

    # Priors for unknown model parameters
    
    # Prior for intercept is still mean of observed - different from prior used
    # in our version
    intercept = pm.Normal('intercept', mu=np.mean(y), sd=1000)
    
    # note shape = 2 parameter
    beta = pm.Normal('beta', mu=0, sd=10, shape=2)
    sigma = pm.Normal('sigma',mu = 1000, sd=50)

    # Expected value of outcome
    mu = intercept + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=y)
    
    # instantiate sampler
    # Slice is a simpler sampler
    step = pm.Slice()

    # draw 500, the 5000 posterior samples
    trace_500_housing = pm.sample(500, step=step)
    trace_5k_housing = pm.sample(5000, step = step)

#### Plotting the distributions of the posteriors, with both 500 and 5,000 draws.

In [ ]:
pm.traceplot(trace_500_housing);

In [ ]:
pm.traceplot(trace_5k_housing);

Printing out a raw observations--  

Just in case you wanted to see what they look like

In [ ]:
list(trace_500_housing.points())[:2]